In [1]:
from netCDF4 import Dataset
import numpy as np
import xarray as xr
from glob import glob
import time
import warnings
import datetime
import pandas as pd

warnings.filterwarnings("ignore",category=RuntimeWarning)

/Users/danstechman/anaconda/lib/python3.5/site-packages/xarray/core/formatting.py:16: FutureWarning: The pandas.tslib module is deprecated and will be removed in a future version.
  from pandas.tslib import OutOfBoundsDatetime


## Plot Settings
This is where the most commonly modified settings reside (i.e., flight, plot start/end times, data location, save path and file type, etc.)

In [2]:
flight = 'RF01_20180116'

# Specify start and end times to include in concatenated dataset
#    Set startT to 'beg' to use the first file in the whole flight directory
#    Set endT to 'end' to use the last file in the whole flight directory
# startT = '20180119_004500'
# endT = '20180119_011500'
startT = 'beg'
endT = 'end'

files = sorted(glob('/Volumes/SOCRATES_1/HCR/cfradial/moments/10hz/' + flight + '/cfrad.*HCR*.nc'))
outFileName = '/Volumes/SOCRATES_1/HCR/cfradial/moments/' + flight + '_concat-HCR-moments.nc'

# The value 12 here may change. Should equal number of negative values at start of range variable (could automate this...)
rGateStrt = 12

## Data File ID
Determine which data files will provide data between the user-defined start and end times

In [3]:
if startT == 'beg':
    strtFIx = 0
else:
    # Search strings designed to grab cfradial files with the desired times
    strtStr = '.' + startT
    
    # Get the index of the first occurence of cfradial file in the data directory 
    #    containing the start string
    strtFIx = ([ix for ix, f in enumerate(files) if strtStr in f])[0]
    
        
if endT == 'end':
    endFIx = len(files)-1
else:
    # Search strings designed to grab cfradial files with the desired times
    endStr = 'to_' + endT
    
    # Get the index of the first occurence of cfradial file in the data directory 
    #    containing the start string
    endFIx = ([ix for ix, f in enumerate(files) if endStr in f])[0]
    


# Create an array of file indices (from our file listing) to loop over
# rFileIx = np.arange(210,220)
# rFileIx = np.arange(0,len(files))
rFileIx = np.arange(strtFIx,endFIx)


In [4]:
totalTime = 0;
for rFile in rFileIx:
    radData = xr.open_dataset(files[rFile])
    totalTime += radData.dims['time']
totalRng = radData.dims['range'] - rGateStrt

print(totalTime)
print(totalRng)

267822
758


In [5]:
time1d_all = np.zeros((totalTime,),dtype='datetime64[ns]')
radElev_all = np.zeros((totalTime,))

alt_all = np.zeros((totalTime,totalRng))
time_all = np.zeros((totalTime,totalRng),dtype='datetime64[ns]')
dbz_all = np.zeros((totalTime,totalRng))
vel_all = np.zeros((totalTime,totalRng))
width_all = np.zeros((totalTime,totalRng))
snr_all = np.zeros((totalTime,totalRng))
dbm_all = np.zeros((totalTime,totalRng))
ncp_all = np.zeros((totalTime,totalRng))
ldr_all = np.zeros((totalTime,totalRng))

## Concatenate Variables Across All Files
Loop through the files covering the requested time period and create numpy arrays containing all of the data in this range.

Also, create modified time and altitude arrays which need to be 2-dimensional for plotting.
<br>The altitude array is also modified using the gate range from the aircraft to yield ground-relative altitudes for each gate.

In [6]:
strtIx = 0
for rFile in rFileIx:
    # Open the current radar data file
    radData = xr.open_dataset(files[rFile])
    
    # Get length of time dimension of current file
    endIx = radData.dims['time'] + strtIx
        
    # Pull out the dimension variables
    gateRange_1d = radData['range'].data[rGateStrt:]
    alt_1d = radData['altitude'].data
    time_1d = radData['time'].data
    
    radElev_1d = radData.elevation.data
    
    # HCR elevation angle is positive when pointing upward
    # Adjust the gateRange variable if this is the case to properly initialize
    #    the ground-relative altitude array
    if np.all(radElev_1d > 0):
        gateRange_1d *= -1
    
    # Create arrays of gateRange and gate altitude matching dimensions of 2-D variables
    gateRange_2d = np.transpose(np.tile(gateRange_1d,(len(time_1d),1)))
    time_2d = np.tile(time_1d,(len(gateRange_1d),1))
    alt_2d = np.tile(alt_1d,(len(gateRange_1d),1))

    
    # Create altitude array indicating the ground-relative altitude MSL of each gate
    alt_all[strtIx:endIx,:] = np.transpose(alt_2d - gateRange_2d)
    time_all[strtIx:endIx,:] = np.transpose(time_2d)
    time1d_all[strtIx:endIx] = time_1d[:]
    radElev_all[strtIx:endIx] = radElev_1d[:]

    dbz_all[strtIx:endIx,:] = radData['DBZ'].data[:,rGateStrt:]
    vel_all[strtIx:endIx,:] = radData['VEL'].data[:,rGateStrt:]
    width_all[strtIx:endIx,:] = radData['WIDTH'].data[:,rGateStrt:]
    snr_all[strtIx:endIx,:] = radData['SNR'].data[:,rGateStrt:]
    dbm_all[strtIx:endIx,:] = radData['DBMHX'].data[:,rGateStrt:]
    ncp_all[strtIx:endIx,:] = radData['NCP'].data[:,rGateStrt:]
    ldr_all[strtIx:endIx,:] = radData['LDR'].data[:,rGateStrt:]
        
        
    strtIx = endIx

## Write concatenated data out to NetCDF

In [7]:
## Create the netCDF file and define global/variable attributes
rootGrp = Dataset(outFileName,'w',format='NETCDF4')
rootGrp.set_fill_on()

In [8]:
gateRng = rootGrp.createDimension('gateRng',totalRng)
time1d = rootGrp.createDimension('time1d',totalTime)

In [9]:
TIME = rootGrp.createVariable('time1d','f8',('time1d',))
TIME.long_name = 'Timestamp'
TIME.units = 'UTC YYYYMMDDTHHMMSS.f'
TIME.coordinates = 'time1d'

GRNG = rootGrp.createVariable('gateRng','f4',('gateRng',),fill_value=np.nan)
GRNG.long_name = 'Distance from radar to center of each range gate'
GRNG.units = 'm'
GRNG.coordinates = 'gateRng'

TIME2D = rootGrp.createVariable('time2d','f8',('time1d','gateRng'))
TIME2D.long_name = 'Timestamp for every gate/time'
TIME2D.units = 'UTC YYYYMMDDTHHMMSS.f'
TIME2D.coordinates = 'time1d gateRng'

GALT2D = rootGrp.createVariable('gateAlt2d','f4',('time1d','gateRng'),fill_value=np.nan)
GALT2D.long_name = 'Altitude of each gate above MSL'
GALT2D.units = 'm'
GALT2D.coordinates = 'time1d gateRng'

DBZ = rootGrp.createVariable('DBZ','f4',('time1d','gateRng'),fill_value=np.nan)
DBZ.long_name = 'reflectivity'
DBZ.units = 'dBZ'
DBZ.sampling_ratio = 1.000000
DBZ.grid_mapping = 'grid_mapping'
DBZ.coordinates = 'time1d gateRng'

VEL = rootGrp.createVariable('VEL','f4',('time1d','gateRng'),fill_value=np.nan)
VEL.long_name = 'doppler_velocity'
VEL.units = 'm/s'
VEL.sampling_ratio = 1.000000
VEL.grid_mapping = 'grid_mapping'
VEL.coordinates = 'time1d gateRng'

WIDTH = rootGrp.createVariable('WIDTH','f4',('time1d','gateRng'),fill_value=np.nan)
WIDTH.long_name = 'spectrum_width'
WIDTH.units = 'm/s'
WIDTH.sampling_ratio = 1.000000
WIDTH.grid_mapping = 'grid_mapping'
WIDTH.coordinates = 'time1d gateRng'

LDR = rootGrp.createVariable('LDR','f4',('time1d','gateRng'),fill_value=np.nan)
LDR.long_name = 'linear_depolarization_ratio'
LDR.units = 'dB'
LDR.sampling_ratio = 1.000000
LDR.grid_mapping = 'grid_mapping'
LDR.coordinates = 'time1d gateRng'

NCP = rootGrp.createVariable('NCP','f4',('time1d','gateRng'),fill_value=np.nan)
NCP.long_name = 'normalized_coherent_power'
NCP.units = ''
NCP.sampling_ratio = 1.000000
NCP.grid_mapping = 'grid_mapping'
NCP.coordinates = 'time1d gateRng'

SNR = rootGrp.createVariable('SNR','f4',('time1d','gateRng'),fill_value=np.nan)
SNR.long_name = 'signal_to_noise_ratio'
SNR.units = 'dB'
SNR.sampling_ratio = 1.000000
SNR.grid_mapping = 'grid_mapping'
SNR.coordinates = 'time1d gateRng'

RELV = rootGrp.createVariable('elevation','f4',('time1d',),fill_value=np.nan)
RELV.long_name = 'Radar beam elevation (positive is upwards [plane-relative])'
RELV.units = 'degrees'
RELV.coordinates = 'time1d'


# Global attributes
rootGrp.description = 'Concatenated HIAPER Cloud Radar data'
rootGrp.flight = flight
rootGrp.history = 'Created ' + time.asctime(time.gmtime()) + ' UTC'
rootGrp.firstFile = files[strtFIx]
rootGrp.lastFile = files[endFIx]

## Write variables to file
TIME[:] = time1d_all.astype(np.float64)
GRNG[:] = gateRange_1d
TIME2D[:] = time_all.astype(np.float64)
GALT2D[:] = alt_all
DBZ[:] = dbz_all
VEL[:] = vel_all
WIDTH[:] = width_all
LDR[:] = ldr_all
NCP[:] = ncp_all
SNR[:] = snr_all
RELV[:] = radElev_all

rootGrp.close()

In [10]:
print(time1d_all.shape)
print(gateRange_1d.shape)
print(time_all.shape)
print(alt_all.shape)
print(dbz_all.shape)
print(vel_all.shape)
print(width_all.shape)
print(ldr_all.shape)
print(ncp_all.shape)
print(snr_all.shape)
print(radElev_all.shape)

(267822,)
(758,)
(267822, 758)
(267822, 758)
(267822, 758)
(267822, 758)
(267822, 758)
(267822, 758)
(267822, 758)
(267822, 758)
(267822,)


In [11]:
time1d_all[0]

numpy.datetime64('2018-01-15T21:51:17.312542000')